# Introduction

(NOTE: This notebook is intended for use with the slides found [here](https://github.com/cropsinsilico/FSPM2020_yggdrasil_workshop/blob/master/slides.pdf)).

This is a Jupyter notebook. It allows us to run code (in this case Python) alongside text in different "cells". This cell is a markdown cell that can display text and html, the next cell is a code cell.

In the code cells (prefixed by `In [ ]:`), you can assign variables, perform calculations or call external functions/classes. You can run code cells by selecting the cell (so that a blue or green box appears around it) and then clicking the run button or holding `Shift+Enter`. Then a number will appear inside the brackets indicating the order of when the cell was executed. 

Output from the cell will be displayed below it with the `Out[#]:` prefix where the number in the brackets indicates the input cell that generated it.

In [ ]:
x = 1
y = 3
z = (x + y)**3
z

The cell below imports external Python code for use here (specifically a package `trimesh` for loading and displaying 3D meshes in the notebook and some utilities for viewing file contents).

In [ ]:
import trimesh
from utils import *
from yggdrasil.runner import run

The cell below uses the `trimesh` package to load an example 3D mesh (plants-2) that will be used in today's exercises. After running this cell, a display of the 3D mesh should appear that you can rotate by clicking and dragging (note that you have to click a second time to stop manipulating the mesh).

In [ ]:
fname = 'meshes/plants-2.obj'
mesh = trimesh.load_mesh(fname)
mesh.show()

# Toy Plant Model

For the exercises today, we will be using a toy functional-structural plant model that makes the above plant mesh grow over time.

In the first version of this model, plant_v0.py (shown below), the growth is very simple and scales directly with time.

In [ ]:
from models import plant_v0
print_python_source(plant_v0.run)

The cell below runs this model directly (it is written in Python) and displays the resulting mesh at the end of the run. As you can see the mesh has 'grown' vertically.

In [ ]:
mesh = plant_v0.run(mesh, 0, 28, 1)
mesh.show()

However, we can also run the model using `yggdrasil` if we have a yaml configuration file for it. The cell below displays the configuration file for this model.

In [ ]:
print_yaml('yamls/plant_v0.yml')

The following code allows yggdrasil to run the model using the Python API. This is usually done via the `yggrun` command line utility (e.g. `yggrun yamls/plant_v0.yml`). Because this method of running the model requires loading the `yggdrasil` library, it takes a bit longer to run, but would not be significantly larger for a model with a more realistic (and computationally intensive) calculation.

In [ ]:
run('yamls/plant_v0.yml')
display_last_timestep()

But now we want to add information about other processes to our plant model...

# Call to Light Model

We can add information by calling other models as functions. For example, consider the toy light model below. It is a function that takes height and time as input and returns the light. 

Height can be a scalar or an array, but it is computationally more efficient to pass the 

In [ ]:
from models import light
print_python_source(light.light)

Because the light model is a function, yggdrasil can automatically wrap it with interface send and receive calls. Therefore, the YAML specification file for this model is relativly simple and the light model itself does not need to be modified at all.

In [ ]:
print_yaml('yamls/light.yml')

Then we can add a interface call to the plant model "client" that calls the light model "server". In addition to the client comm, I have also added an output comm for the light values so that we can plot the light values.

In [ ]:
from models import plant_v1
print_python_source_diff(plant_v0.run, plant_v1.run)

The YAML specification file for this version of the plant model differs in that it specifies that it is a client of the model with the name "light" and that the light output should be directed to a file if it is not connected to a model.

In [ ]:
print_yaml_diff('yamls/plant_v0.yml', 'yamls/plant_v1.yml')

We can run the integration of these two models by providing the paths to the YAML specification models to the yggdrasil API. The command to run this integration on the command line would be `yggrun yamls/plant_v1.yml yamls/light.yml`.

In [ ]:
run(['yamls/plant_v1.yml', 'yamls/light.yml'], production_run=True)

The cell below displays the resulting mesh with color mapped to the light. We can see that the growth tracked by this version of the model is much more subtle.

In [ ]:
display_last_timestep(with_light=True)

## The light model is easily replaced with other models

Feel free to try running one or more of them. They have been coded to produce the same results as the Python version of the light model.

### C++ Model

In [ ]:
print_yaml('yamls/light_cpp.yml')
print_source('models/light.cpp')
run(['yamls/plant_v1.yml', 'yamls/light_cpp.yml'], production_run=True)
display_last_timestep(with_light=True)

### R Model

In [ ]:
print_yaml('yamls/light_R.yml')
print_source('models/light.R')
run(['yamls/plant_v1.yml', 'yamls/light_R.yml'], production_run=True)
display_last_timestep(with_light=True)

# Time Step Synchronization

In addition to allowing models to call other models as functions, yggdrasil also provides a method of synchronizing data across time dependent models, even if the models have different time steps. To see how this works in practice, the following shows how we could connect the plant model from above with the time-dependent root growth model shown below.

In [ ]:
from models import roots_v0
print_python_source(roots_v0.run)

To add time step synchonization, we add a `timesync` comm to both the plant and root models. In the updated version of the plant model displayed by the cell below, there is now a `YggTimesync` comm that can be called using the same syntax as the `YggRpcClient` to get data from other time dependent models inside the time loop (in this case a root growth model).

In [ ]:
from models import plant_v2
print_python_source_diff(plant_v1.run, plant_v2.run)

Similarly, a `YggTimesync` comm is also added to the root model with calls inside the time loop. In addition to the time step synchronization comm, an output comm is also added to the root model so we can output masses to a file.

In [ ]:
from models import roots_v1
print_python_source_diff(roots_v0.run, roots_v1.run)

And the yaml specification file for tha plant model now includes a `timesync` parameter specifying the name of the timestep synchronization dummy model that should be used.

In [ ]:
print_yaml_diff('yamls/plant_v1.yml', 'yamls/plant_v2.yml')

The yaml file for the roots model contains the same value for the `timesync` parameter (indicating they should share data) and an output for the calculated masses.

In addition, there is an explicit entry for the named `timesync` "model" that specifies that the data exchanged should be aggregated by summing.

In [ ]:
print_yaml('yamls/roots.yml')

This integration can then be run by providing the models for the updated plant model, light model, and root model to the yggdrasil `run` method.

In [ ]:
run(['yamls/plant_v2.yml', 'yamls/light.yml', 'yamls/roots.yml'], production_run=True)
display_last_timestep(with_light=True)
plot_mass()

# Some Notes

## `production_run` Keyword
You may have noticed that we have been passing the `production_run` keyword to the `run` API function with a value of `True`. When set to `True`, yggdrasil turns of several safe guards that increase run-time. These include things like checking data formats and validating inputs/outputs to/from framework components. It is highly recommended, that `production_run` is only set to `True` when you are done testing an integration and are ready for a "production run" that requires higher performance.

## Splitting Calls
As you may remember from the introduction, client and timesync "calls" are really a combination of a send and receive (sending the request and receiving the response). If call's take a long amount of time, we can split the call into its components to take advantage of the parallelism yggdrasil offers. A client or timesync comm can call send, finish another unrelated task while the server/sync operation takes place, and then call receive to get the response. For example, the synchronization call to the root model in the previous example could be split between a send and receive call::

In [ ]:
from models import plant_v2_split
print_python_source_diff(plant_v2.run, plant_v2_split.run)

# Additional Demos

## Simple input/output

The previous techniques are what I believe will be the most useful for using yggdrasil with functional, structural models, but yggdrasil also supports more atomic communication patterns that can be used to build up complex ones. Below, I have added an output comm to the original plant model to output meshes rather than save them directly using trimesh.

In [ ]:
from models import plant_output_mesh
print_python_source_diff(plant_v0.run, plant_output_mesh.run)

The yaml for such a model is shown below. It now includes an `outputs` section listing the new output channel and defining a `default_file` which is where outputs sent to that channel will be saved if the output channel is not connected to an input channel.

In [ ]:
print_yaml_diff('yamls/plant_v0.yml', 'yamls/plant_output_mesh.yml')

The same can be done for inputs. The version of the plant model shown by the cell below includes an input and an output for the mesh at each timestep.

In [ ]:
from models import plant_io_mesh
print_python_source_diff(plant_output_mesh.run, plant_io_mesh.run)

And the yaml for this model now includes an `inputs` section as well as `connections` section connecting the output from the `plant_output_mesh` version of the plant model to version with input and output (named `plant2`).

In [ ]:
print_yaml_diff('yamls/plant_output_mesh.yml', 'yamls/plant_io_mesh.yml')

When this integration is run in the cell below, it passes output from one model to the next at each time step.

In [ ]:
run(['yamls/plant_output_mesh.yml', 'yamls/plant_io_mesh.yml'], production_run=True)
display_last_timestep()